In [ ]:
import torch
import torchvision
from torch.utils import data
from torchvision import transforms

from torch import nn


### 数据处理

In [2]:
batch_size = 256

trans = transforms.ToTensor()
# 创建Dataset对象
mnist_train = torchvision.datasets.FashionMNIST(root="../data/",train=True,transform=trans,download=True)
mnist_test = torchvision.datasets.FashionMNIST(root="../data/",train=False,transform=trans,download=True) 

train_iter = data.DataLoader(mnist_train, batch_size, shuffle=True, num_workers=4) # 加载数据并分批
test_iter = data.DataLoader(mnist_train, batch_size, shuffle=True, num_workers=4)

### 初始化

In [3]:
net = nn.Sequential(nn.Flatten(),nn.Linear(784,256),nn.ReLU(),nn.Linear(256,10))

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

net.apply(init_weights)

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=256, bias=True)
  (2): ReLU()
  (3): Linear(in_features=256, out_features=10, bias=True)
)

### 训练

In [6]:
batch_size, lr, num_epochs = 256, 0.1, 10
loss = nn.CrossEntropyLoss()
trainer = torch.optim.SGD(net.parameters(), lr=lr)

num_epochs = 10
num_correct = 0
train_loss = 0
num_samples = 0

for epoch in range(num_epochs):
    for X, y in train_iter:  # 从DataLoader里面一次一次把所有数据拿出来
        l = loss(net(X),y) # net(X) 为计算出来的线性回归的预测值
        trainer.zero_grad() # 梯度清零
        l.backward()
        trainer.step()  # SGD优化器优化模型
    print(f'epoch:{epoch+1},cost:{l:f}')

In [7]:
test_loss = 0
num_samples = 0
num_correct = 0
with torch.no_grad():
    for X, y in test_iter:
        out = net(X)
        batch_loss = loss(out, y)
        test_loss += batch_loss.item() * X.shape[0]
        num_samples += X.shape[0]
        # 计算预测类别
        pred = out.argmax(dim=1)
        num_correct += (pred == y).sum().item()
    avg_test_loss = test_loss / num_samples
    accuracy = num_correct / num_samples
print(f"测试集平均loss: {avg_test_loss:.6f}")
print(f"测试集准确率: {accuracy:.4f}")

测试集平均loss: 0.352948
测试集准确率: 0.8731
